# Train/predict using CGCNN with CGCNN

In [ ]:
%env CUDA_VISIBLE_DEVICES=1

In [ ]:
import sys
sys.path.insert(0, '/home/nianhant/script')

import pdb
import os
import csv
import shutil
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn import linear_model as lm
from sigopt import Connection
conn = Connection(client_token="ZSKVODQFWHLZOWZASSFHZIGLIZRXCPIJMQTRVZITZRNPHVYO")

from modify_data import CIFData
import run_cgcnn as rc
import analyze_cgcnn as ac

import pprint

## Manage inputs: optimized experiment, root_dir, train/valid/test size

In [ ]:
# optimized experiment
sigopt_exp_id = 50929
experiments = conn.experiments(id=sigopt_exp_id).fetch()
assignments = conn.experiments(id=sigopt_exp_id).best_assignments().fetch().data[0].assignments
print(assignments)

# database directory
main_dir = '/home/demo/'
root_dir = os.path.join(main_dir, 'root_dir')
assert(os.path.isdir(root_dir))    # double check root_dir exists!

# use ratio for size or mannually defined instead
files = [f for f in os.listdir(root_dir) if f.endswith('.cif')]
data_size = len(files)
train_size, val_size = round(data_size*0.6), round(data_size*0.2)
test_size = data_size - train_size - val_size
print(data_size, train_size, val_size, test_size)

## train, predict
CGCNN only gives output for test set. What I do is use model_best.pth.tar on whole dataset again and find train/valid set output

In [ ]:
move_cgcnn_files = ac.move_cgcnn_files
save_convergence = ac.save_convergence

# load data
print('----Loading data to dataset----')
dataset = CIFData(root_dir)

# train model, notice dataset or datalist used in function
print('----Train model----')
run_model = rc.run_model
epochs, train_mae_errors, train_losses, val_mae_errors, val_losses, test_mae, test_loss = \
    run_model(assignments, root_dir, dataset, train_size, val_size, test_size, evaluate=False)
# save convergence results
# save training data to convergence.csv
save_convergence(epochs, train_mae_errors, train_losses, val_mae_errors, val_losses)

# predict model
print('----Predict model----')
predict_model = rc.predict_model
predict_model(modelpath='model_best_train_%d.pth.tar'%train_size, cifpath=root_dir)    

In [ ]:
# move files to desired folder
import os
print('----Move files----')
source = '/home/demo/'
dst = 'train_predict_result/20181022/'
destination = os.path.join(source, dst)
files = ['checkpoint_train_%d.pth.tar'%train_size, 
         'model_best_train_%d.pth.tar'%train_size, 
         'test_results_train_%d.csv'%train_size, 
         'test_results_predict.csv',
         'convergence_mae_loss.csv']
move_cgcnn_files(files, source, destination)